In [ ]:
SET GITHUB_SECRET_USERNAME = 'ketki-mude';
SET GITHUB_SECRET_PASSWORD = 'xxxx';
SET GITHUB_URL_PREFIX = 'https://github.com/ketki-mude';
SET GITHUB_REPO_ORIGIN = 'https://github.com/ketki-mude/FRED_Currency_Exchange_Pipeline.git';

In [ ]:
USE ROLE ACCOUNTADMIN; 

SET MY_USER = CURRENT_USER();
CREATE OR REPLACE ROLE FRED_ROLE;
GRANT ROLE FRED_ROLE TO ROLE SYSADMIN;
GRANT ROLE FRED_ROLE TO USER IDENTIFIER($MY_USER);

GRANT EXECUTE TASK ON ACCOUNT TO ROLE FRED_ROLE;
GRANT MONITOR EXECUTION ON ACCOUNT TO ROLE FRED_ROLE;
GRANT IMPORTED PRIVILEGES ON DATABASE SNOWFLAKE TO ROLE FRED_ROLE;

-- Databases
CREATE OR REPLACE DATABASE FRED_DB;
GRANT OWNERSHIP ON DATABASE FRED_DB TO ROLE FRED_ROLE;

-- Warehouses
CREATE OR REPLACE WAREHOUSE FRED_WH WAREHOUSE_SIZE = XSMALL, AUTO_SUSPEND = 300, AUTO_RESUME= TRUE;
GRANT OWNERSHIP ON WAREHOUSE FRED_WH TO ROLE FRED_ROLE;


In [ ]:
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE FRED_WH;
USE DATABASE FRED_DB;

-- Schemas
CREATE OR REPLACE SCHEMA INTEGRATIONS;


CREATE OR REPLACE SECRET DEMO_GITHUB_SECRET
  TYPE = password
  USERNAME = $GITHUB_SECRET_USERNAME
  PASSWORD = $GITHUB_SECRET_PASSWORD;

CREATE OR REPLACE API INTEGRATION DEMO_GITHUB_API_INTEGRATION
  API_PROVIDER = GIT_HTTPS_API
  API_ALLOWED_PREFIXES = ($GITHUB_URL_PREFIX)
  ALLOWED_AUTHENTICATION_SECRETS = (DEMO_GITHUB_SECRET)
  ENABLED = TRUE;

-- Git Repository
CREATE OR REPLACE GIT REPOSITORY FRED_GIT_REPO
  API_INTEGRATION = DEMO_GITHUB_API_INTEGRATION
  GIT_CREDENTIALS = DEMO_GITHUB_SECRET
  ORIGIN = $GITHUB_REPO_ORIGIN;

In [ ]:
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE FRED_WH;
USE DATABASE FRED_DB;
CREATE OR REPLACE SCHEMA EXTERNAL;

CREATE OR REPLACE STORAGE INTEGRATION fred_s3_integration
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = 'S3'
  ENABLED = TRUE
  STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::699475925561:role/snowflake_s3_role'
  STORAGE_ALLOWED_LOCATIONS = ('s3://fredcurrencyexhange/');
GRANT USAGE ON INTEGRATION fred_s3_integration TO ROLE FRED_ROLE;
GRANT USAGE ON SCHEMA EXTERNAL TO ROLE FRED_ROLE;
GRANT ALL PRIVILEGES ON SCHEMA FRED_DB.EXTERNAL TO ROLE FRED_ROLE;

-- Create the file format
USE ROLE FRED_ROLE;
USE WAREHOUSE FRED_WH;
USE DATABASE FRED_DB;
USE SCHEMA EXTERNAL;

CREATE OR REPLACE FILE FORMAT CSV_FORMAT 
TYPE = 'CSV' 
FIELD_OPTIONALLY_ENCLOSED_BY = '"' 
PARSE_HEADER = TRUE;


In [ ]:
USE ROLE ACCOUNTADMIN;
USE WAREHOUSE FRED_WH;
USE SCHEMA FRED_DB.INTEGRATIONS;

EXECUTE IMMEDIATE FROM @FRED_GIT_REPO/branches/main/scripts/noteboook_deploy.sql
    USING (env => 'DEV', branch => 'main')